<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# 结构化输出

In [ ]:
from videos.walkthroughs import walkthrough_41 as walkthrough

In [ ]:
walkthrough()

在这个 notebook 中，我们将介绍如何使用 LLM 生成结构化输出，并探索一些批量生成下游数据的方法。

---

## 目标

完成这个 notebook 后，您将：

- 了解让 LLM 生成结构化输出的价值。
- 提示您的模型生成结构化输出。
- 使用聊天模型将输入批量处理为结构化数据。

---

## 导入

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, SimpleJsonOutputParser
from langchain_core.runnables import RunnableLambda

---

## 创建模型实例

In [ ]:
base_url = 'http://llama:8000/v1'
model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(base_url=base_url, model=model, temperature=0)

---

## LLM 和高度结构化的数据格式

我们希望 LLM 执行的一个非常常见的任务是以高度结构化的格式生成输出。这些格式可以是常见的 JSON，或是 Python 列表，或者是一些根据我们需求定制的结构，比如自定义报告或文档结构。

LLM 的表现在逐渐变好，它们生成高度结构化数据的能力大幅提升，即使是小型的 LLM（比如今天使用的 8B 模型），也可以通过一些提示工程，以使模型能够持续产生我们所需的内容，比如 JSON（或任何类型的代码）或其它特定格式的结构。

让我们来处理一个非常常见的任务：让模型生成结构化的 JSON。JSON 在许多应用场景中都是一个很好的结构，因为它可以直接用于许多后续任务，或者将 JSON 转换为各种其它可用格式，如 Python 字典、dataframes 等等。

---

## 一个简单的 JSON 对象

继续迭代提示词的思路，先从简单开始，设计一个提示词来指导模型构建一个 JSON 对象。这里我们要求模型创建一个简单的 JSON 对象，表示圣克拉拉市的详细信息。

In [ ]:
prompt = '''\
Make a JSON object representing the city Santa Clara. \
It should have fields for: \
- The name of the city \
- The country the city is located in.'''

In [ ]:
print(llm.invoke(prompt).content)

我们收到了模型返回的一些对话文本，这些文本并不是我们想要的，但在响应中有一个看起来非常不错的 JSON 对象，这很好。

顺便说一下，LLM 在生成结构化输出方面的能力正在迅速提高，我们期待它们会变得越来越好。甚至在几个月前（本文于 2024 年夏天撰写），使用 Llama 3.1 的前身 Llama 2，用如此简单的提示从一个 8B 模型中得到这样好的响应基本是不可能的。

我们仍然有工作要做，下面就来迭代一下提示词，看看能否去掉这些对话文本。

In [ ]:
prompt = '''\
Make a JSON object representing the city Santa Clara. \
It should have fields for:
- The name of the city
- The country the city is located in.

Only return the JSON. Never return non-JSON text.'''

In [ ]:
print(llm.invoke(prompt).content)

现在更接近我们的目标了，接下来，看看能否继续去掉引号的包裹。

In [ ]:
prompt = '''\
Make a JSON object representing the city Santa Clara. \
It should have fields for:
- The name of the city
- The country the city is located in.

Only return the JSON. Never return non-JSON text including backtack wrappers around the JSON.'''

In [ ]:
print(llm.invoke(prompt).content)

这才是我们想要的。下面把模型响应加载到一个 Python 字典，遍历它来验证一下。

In [ ]:
json_city = llm.invoke(prompt).content

In [ ]:
import json
python_city = json.loads(json_city)

for k, v in python_city.items():
    print(f'{k}: {v}')

---

## 将提示词制作成模板

接下来，让我们把提示词转换为一个提示模板，以便将城市名称参数化。

In [ ]:
json_city_template = ChatPromptTemplate.from_template('''\
Make a JSON object representing the city {city_name}. \
It should have fields for:
- The name of the city
- The country the city is located in.

Only return the JSON. Never return non-JSON text including backtack wrappers around the JSON.''')

接下来，我们将组合一个简单的链。

In [ ]:
parser = StrOutputParser()

In [ ]:
chain = json_city_template | llm | parser

In [ ]:
print(chain.invoke({'city_name': 'Santa Clara'}))

这看起来也不错。

---

## 简单的 JSON 解析

为了确认我们可以将 JSON 对象加载为 Python 字典，这里用一个自定义的运行时直接将模型响应解析为 Python 字典。

In [ ]:
parse_to_dict = RunnableLambda(lambda response: json.loads(response.content))

重新组合我们的链，以使用这个自定义解析器。

In [ ]:
chain = json_city_template | llm | parse_to_dict

In [ ]:
chain.invoke({'city_name': 'Santa Clara'})

这看起来不错。

再加一个小改进，LangChain 已经提供了 `SimpleJsonOutputParser` 来处理这种情况。让我们用它重建链。

In [ ]:
from langchain_core.output_parsers import SimpleJsonOutputParser

In [ ]:
json_parser = SimpleJsonOutputParser()

In [ ]:
chain = json_city_template | llm | json_parser

In [ ]:
chain.invoke({'city_name': 'Santa Clara'})

---

## 批量处理多个输入

到目前为止一切顺利，出于迭代提示词开发的原则，我们再在几个不同的输入上测试这个链。

In [ ]:
city_names = [
    {'city_name': 'Santa Clara'},
    {'city_name': 'Busan'},
    {'city_name': 'Cairo'},
    {'city_name': 'Perth'}
]

In [ ]:
city_details = chain.batch(city_names)

In [ ]:
city_details

In [ ]:
for city in city_details:
    print(f'City: {city['name']}\nCountry: {city['country']}\n')

---

## 结构与生成

我们在整个课程中一直在用 LLM 生成内容，虽然可能很明显，但还是值得强调一下：我们不仅是让 LLM 结构化给定数据，同时也结合了它的生成能力。

在刚刚处理的例子中，输入的数据是一个城市名称，我们希望将其结构化成 JSON。但不仅是结构化这个信息（城市的名称），我们还利用了模型的生成能力扩展数据内容，加入了城市所在的国家，而这些信息我们并没有直接提供给模型。

结合 LLM 生成能力生成结构化输出/数据，真的非常强大。

---

## 练习：生成书籍详细信息列表

使用到目前为止您学到的技术，生成一个包含字典的 Python 列表，每个字典都包含以下书籍详细信息。

每个字典应包含书名、作者和原始出版年份。

如果您遇到困难，可以随时查看下面的*参考答案*。

In [ ]:
sci_fi_books = [
    {"book_title": "Dune"},
    {"book_title": "Neuromancer"},
    {"book_title": "Snow Crash"},
    {"book_title": "The Left Hand of Darkness"},
    {"book_title": "Foundation"}
]

### 您的代码

### 参考答案

In [ ]:
book_template = ChatPromptTemplate.from_template('''\
Make a JSON object representing the details of the following book: {book_title}. \
It should have fields for:
- The title of the book.
- The author of the book.
- The year the book was originally published.

Only return the JSON. Never return non-JSON text including backtack wrappers around the JSON.''')

In [ ]:
chain = book_template | llm | json_parser

In [ ]:
chain.batch(sci_fi_books)

---

## 总结

在这个 notebook 中，您开始接触 LLM 生成结构化输出的能力。下一个 notebook 您将使用 Pydantic 类和 LangChain 的 JsonOutputParser 来大幅提升这项能力。